# Brief Overview of the `bxilog` Python module

The `bxilog` python module has the following properties:

* [high performance](performance.ipynb)
* [easy to use](usage.ipynb)
* [full-featured](features.ipynb)
* [minimum compliance with standard Python logging module](compat.ipynb);
    
# Main purpose: no print or printf statements

The `bxilog` python module has been designed to remove all print or printf statements. The basic idea is that any message a thread outputs might be able to go to various location according to the configuration.

# Logger, Level, Filter and Handler concepts

The `bxilog` python module provides different kinds of objects.

## Logger

A logger instance must be used in order to produce a log. A logger can safely be traversed by multiple threads concurrently. A logger is given a name. For example:

In [1]:
%%python

import bxi.base.log as bxilog

logger = bxilog.get_logger("my.stuff")
logger.out("Something to say")
logger.warn("Hey!")

Something to say


[W] my.stuff     Hey!


The `bxilog` python module provides a default logger with the empty name: ''. It can be used directly as in the following:

In [33]:
%%python

import bxi.base.log as bxilog

bxilog.out("Something said with the default logger")

Something said with the default logger


From now, you see little benefits from using `bxilog` instead of the standard `print` statement. This will start to appear when we will consider the **level** concept.

## Level

Each logger is given a level which tells if the log must be produced or not at the thread level.

In [40]:
%%python

import bxi.base.log as bxilog

names = bxilog.LEVEL_NAMES
bxilog.out("Level names: %s", ",".join(names))
logger = bxilog.get_logger("my.stuff")
logger.out("Level for %s: %s", logger.name, names[logger.level])


Level names: off,panic,alert,critical,error,warning,notice,output,info,debug,fine,trace,lowest
Level for my.stuff: lowest


As you see the level for the new logger is `lowest`. This basically means that all logging functions made through this logger actually emits a log. This does not mean however that the log will end in the output as shown below:

In [43]:
%%python

import bxi.base.log as bxilog

logger = bxilog.get_logger("my.stuff")
bxilog.debug("Something to say")

The reason for this is handlers and their filters.

## Handlers and Filters

Actually, when a log is emmited by a thread through a logger, it is received by handlers. A handler is responsible for the production of the log in a given format "somewhere": it can be a file, the standard output, the standard error, the network or anything else. Handlers can be plugged in. By default, a console handler is provided. This means that log emmitted goes either to the standard output or to the standard error according to the level and to the filters related to the handler. A filter is a simple pair of (`prefix`, `level`): if a log has been emmitted by a logger with a name that matches the filter's `prefix` at a level above the filter's `level`, then it is produced.
By default, the console handler only produces logs on the standard output for level `lowest` to `warning`, and on the standard error for level `error` to `panic`. The example below will make it clearer.

In [48]:
%%python

import bxi.base.log as bxilog


config = {'handlers': ['console'],
          'setsighandler': True,
          'console': {
                      'module': 'bxi.base.log.console_handler',
                      'filters': 'my:debug',
                      'stderr_level': 'WARNING',
                      'colors': 'none',
                      }
          }
bxilog.set_config(config)

logger = bxilog.get_logger("my.stuff")
logger.out("Something normal to say")
logger.alert("Shouting loud!")
logger.debug("Not that important!")
logger.lowest("Murmuring, don't pay attention...")


Something normal to say
[D] my.stuff     Not that important!


[A] my.stuff     Shouting loud!


As you see, the log at `lowest` level has not been produced. This is because the filters given to the console handler in the configuration specified 'my:debug' tells: "logs emmitted by a logger with a name starting with `my` and at the level `debug` or above must be produced". 

Note that if you want all logs emmitted at the `debug` level, you will specifies filters with the following pattern: `:debug`. The empty prefix '' will match any logger name (any logger name starts with the empty string). 
Try it and see what happens.

## Configuration

Configuration of the BXI logging library can be done through a dictionnary, a `ConfigObj` or a file.

Basically, you specify a list of `handlers` and for each of them, you specify its configuration. Its configuration always include the python module name, and the filters. Most handlers requires other parameters. You must have a look to the documentation to know the list of required parameters.

The configuration below shows a good example for production use where all logs emmitted at the debug level are produced in a file while the console only gets normal output level logs and error logs.

In [9]:
%%python

import bxi.base.log as bxilog
import syslog
import configobj

config = {'handlers': ['console', 'file', ],
          'setsighandler': True,
          'console': {
                      'module': 'bxi.base.log.console_handler',
                      'filters': ':output',
                      'stderr_level': 'WARNING',
                      'colors': 'none',
                      },
          'file': {
                      'module': 'bxi.base.log.file_handler',
                      'filters': ':debug',
                      'path': '/tmp/foo.log',
                      'append': False,
                  },
    }
    
bxilog.set_config(configobj.ConfigObj(config))

logger = bxilog.get_logger("my.stuff")
logger.out("Something normal to say")
logger.alert("Shouting loud!")
logger.debug("Not that important!")
logger.lowest("Murmuring, don't pay attention...")



Something normal to say


[A] my.stuff     Shouting loud!


## Logging and exceptions

When an exception happens, two things might happen:

    * someone in the stack catches the exception and deals with it;
    * no one catches the exception and the thread exits.
    
In the first case, the correct way to report an exception is to log it as shown below:

In [2]:
%%python

import bxi.base.log as bxilog
import configobj

config = {'handlers': ['console', 'file', ],
          'setsighandler': True,
          'console': {
                      'module': 'bxi.base.log.console_handler',
                      'filters': ':out',
                      'stderr_level': 'WARNING',
                      'colors': 'none',
                      },
          'file': {
                      'module': 'bxi.base.log.file_handler',
                      'filters': ':trace',
                      'path': '/tmp/foo.log',
                      'append': False,
                  },
    }
bxilog.set_config(configobj.ConfigObj(config))

try:
    raise ValueError("Something wrong happened!")
except ValueError as ve:
    bxilog.exception("Dealing with the exception '%s'", ve.__class__.__name__)

[E]              Dealing with the exception 'ValueError'
[E]              ValueError: Something wrong happened!


Note how the file contains much more information: in particular it holds the traceback!

In [3]:
%%bash

cat /tmp/foo.log

D|20170221T092549.847922227|03765.03765=01792:|log.c:175@bxilog_init|~bxilog|Initialization done
F|20170221T092549.848382090|03765.03765=01792:|signal.c:76@bxilog_install_sighandler|~bxilog.signal|Alternate signal stack set at 0x95a220 (8192 B)
D|20170221T092549.848389341|03765.03765=01792:|signal.c:81@bxilog_install_sighandler|~bxilog.signal|Setting signal handler process wide
D|20170221T092549.848395565|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 11: Segmentation fault
D|20170221T092549.848398965|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 7: Bus error
D|20170221T092549.848401179|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 8: Floating point exception
D|20170221T092549.848402919|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 4: Illegal instruction
D|20170221T092549.848404477|03765.03

In the second case, there is nothing to do, the BXI logging system will take care of the  reporting itself:

In [1]:
%%python

import bxi.base.log as bxilog
import configobj

config = {'handlers': ['console', 'file', ],
          'setsighandler': True,
          'console': {
                      'module': 'bxi.base.log.console_handler',
                      'filters': ':out',
                      'stderr_level': 'WARNING',
                      'colors': 'none',
                      },
          'file': {
                      'module': 'bxi.base.log.file_handler',
                      'filters': ':trace',
                      'path': '/tmp/foo.log',
                      'append': False,
                  },
    }          
bxilog.set_config(configobj.ConfigObj(config))

raise ValueError("Something wrong happened!")

Traceback (most recent call last):
  File "<stdin>", line 22, in <module>
ValueError: Something wrong happened!


In [2]:
%%bash 

cat /tmp/foo.log


D|20170220T192537.423333547|09625.09625=63232:|log.c:175@bxilog_init|~bxilog|Initialization done
F|20170220T192537.423810648|09625.09625=63232:|signal.c:76@bxilog_install_sighandler|~bxilog.signal|Alternate signal stack set at 0x958460 (8192 B)
D|20170220T192537.423817225|09625.09625=63232:|signal.c:81@bxilog_install_sighandler|~bxilog.signal|Setting signal handler process wide
D|20170220T192537.423830696|09625.09625=63232:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 11: Segmentation fault
D|20170220T192537.423833906|09625.09625=63232:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 7: Bus error
D|20170220T192537.423835272|09625.09625=63232:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 8: Floating point exception
D|20170220T192537.423837617|09625.09625=63232:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 4: Illegal instruction
D|20170220T192537.423839200|09625.09

# Parsing files produced by bxilog file handler

The `bxibase` package comes with a file parser that is able to parse a file produced by a bxilog file handler in such a way that:

* log level are highlighted;
* log produced by different threads are highlighted;
* logs are re-ordered when required.

See below for a concrete example.

In [4]:
%%bash

bxilog-parser /tmp/foo.log

D|20170221T092549.847922227|03765.03765=01792:|log.c:175@bxilog_init|~bxilog|Initialization done
F|20170221T092549.848382090|03765.03765=01792:|signal.c:76@bxilog_install_sighandler|~bxilog.signal|Alternate signal stack set at 0x95a220 (8192 B)
D|20170221T092549.848389341|03765.03765=01792:|signal.c:81@bxilog_install_sighandler|~bxilog.signal|Setting signal handler process wide
D|20170221T092549.848395565|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 11: Segmentation fault
D|20170221T092549.848398965|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 7: Bus error
D|20170221T092549.848401179|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 8: Floating point exception
D|20170221T092549.848402919|03765.03765=01792:|signal.c:102@bxilog_install_sighandler|~bxilog.signal|Signal handler set for 4: Illegal instruction
D|20170221T092549.848404477|03765.03